# Notebook 5.1 - Curation-Keywords (alpha release)

The final release of this notebook will implement the workflow defined in:

[Curating keywords](https://gitlab.gwdg.de/sshoc/marketplace-curation/-/issues/1#note_71056) GitLab issue

The notebook works as follows:

0. Imports external libraries and loads the MP dataset and the google sheet
1. Looks for vocabulary terms with the value from the column *Keyword to map*
2. Looks for the term in the column *Map to*
3. Goes through all MP items to look for the *keywords-to-map* in the keyword-dynamic-property
4. Replaces the  the *keywords-to-map* in the MP dataset.


## 0 Requirements to run this notebook

This section gives all the relevant information to "interact" with the MP data.

### 0.1 libraries
*There are a number of external libraries needed to run the notebook* 

*Furthermore, a dedicated SSH Open Marketplace library - sshmarketplacelib - with customised functions has been created and can be imported using the python import commands.* 

*Below the libraries import needed to run this notebook*

In [1]:
import numpy as np
import pandas as pd
import requests
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

### 0.2 Get the data



Get the MarketPlace dataset

In [2]:
mpdata = mpd()
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)
df_trainingmaterials_flat =mpdata.getMPItems ("trainingmaterials", True)
df_workflows_flat =mpdata.getMPItems ("workflows", True)
df_datasets_flat =mpdata.getMPItems ("datasets", True)

getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...


Get the list of keywords from the [gsheet](https://docs.google.com/spreadsheets/d/1-Oh9_SxIhfMAT6KNJrMf4LetCpy5s1fHZEyTL__TUVA/edit#gid=0)

In [3]:
sheet_id = '1-Oh9_SxIhfMAT6KNJrMf4LetCpy5s1fHZEyTL__TUVA'
sheet_name = 'Mappings'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_keywords=pd.read_csv(url)

The function *getMPConcepts()* is a custum function that uses the API entry: 

GET https://marketplace-api.sshopencloud.eu/api/concept-search?perpage=100&q=URI

to get all the *concepts* from the MarketPlace dataset. 

**Note that executing this function may require some time, currently 14995 records are returned**



In [4]:
df_concepts=mpdata.getMPConcepts()

In [5]:
utils=hel.Util()
resultfields=['persistentId', 'MPUrl', 'category', 'label', 'type.code', 'type.label', 'concept.code', 'concept.label', 'concept.uri', 'concept.vocabulary.scheme']
udf_alprop=utils.getAllPropertiesBySources()
udf_alprop=udf_alprop.loc[ : ,resultfields]

A few lines of the gsheet

In [6]:
df_keywords.head()

,Keyword to map,Map to,Comment
0,Linguistics,https://vocabs.acdh.oeaw.ac.at/oefosdiscipline...,Sprach- und Literaturwissenschaften
1,History,https://vocabs.acdh.oeaw.ac.at/oefosdiscipline...,"Geschichte, Archäologie"
2,Literature,https://vocabs.acdh.oeaw.ac.at/oefosdiscipline...,Sprach- und Literaturwissenschaften
3,Video,https://vocabs.sshopencloud.eu/vocabularies/me...,video
4,Text,https://vocabs.sshopencloud.eu/vocabularies/me...,text


## Update the MP dataset

The function *getMPKeywordProperies(mKey)* is a custum function that uses the API entry 

    GET https://marketplace-api.sshopencloud.eu/api/concept-search?types=keyword&q=VALUE

and returns the vocabulary terms for *mKey*.  

In the cell below, the returned dataset is filtered to individuate those values coming from the vocabulary *sshoc-keyword* (vocabulary[code]=sshoc-keyword), then the resulting dataset is shown.


In [7]:

rown=0
noind=[54,62,127]
selectedItems=pd.DataFrame()
for rown, row in df_keywords.iterrows():
    if (rown in noind):
        continue;
    myKey=df_keywords.iloc[rown]['Keyword to map']
    
    print(f'\n***** Checking {myKey}')
    df_vocterms=mpdata.getMPKeywordProperties(myKey)
    
    if (df_vocterms.empty):
        print (f"vvvvv No vocabulary terms found for {myKey}")
        continue;
    df_vocterms=df_vocterms.loc[df_vocterms.vocabulary=={'code': 'sshoc-keyword'}]
    #The set of vocabulary terms is filterd to individuate those with the exact match (case-insensitive) 
    df_vocterms=df_vocterms.loc[(df_vocterms.code).str.lower()==myKey.lower()]

    #Search for the one concept that has in the uri column the value of the Map to. There should be only one such concept.

    test3=df_concepts.loc[df_concepts.uri==df_keywords.iloc[rown]['Map to']]

    #Filter the dataset using the Keyword to Map

    test=udf_alprop.loc[(udf_alprop['concept.label'].str.lower()==myKey.lower()), ]#& (udf_alprop['type.code'].str.lower()=='keyword')]
    
    if (test.empty):
        print (f"iiiii  No Items found for {myKey}")
        continue;
    print (f'fffff  Found as {test.iloc[0]["type.code"]}')
    #update the MP
    jsonConcept={}
    jsonConceptVal={}
    jsonConceptVal["code"]=test3.iloc[0].code
    jsonConceptVal["vocabulary"]=test3.iloc[0].vocabulary
    jsonConceptVal["uri"]=test3.iloc[0].uri
    #jsonConcept
    attrList={}
    attrList["type"]=test3.iloc[0].types[0]
    attrList["concept"]=jsonConceptVal
    filterList={}
    filterList["concept"]=myKey.lower()
    test.loc[ : ,'updateList']=[attrList for _ in range(test.shape[0])]
    test.loc[ : ,'filterList']=[filterList for _ in range(test.shape[0])]
    selectedItems=selectedItems.append(test)

attrList={}
filterList={}
mpdata.updateItemsProperties(selectedItems)



***** Checking Linguistics
fffff  Found as keyword

***** Checking History


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


fffff  Found as keyword

***** Checking Literature
fffff  Found as keyword

***** Checking Video
fffff  Found as object-format

***** Checking Text
fffff  Found as object-format

***** Checking Publishing
fffff  Found as keyword

***** Checking Capture
fffff  Found as keyword

***** Checking Gathering
fffff  Found as keyword

***** Checking Dissemination
fffff  Found as keyword

***** Checking Storage
fffff  Found as keyword

***** Checking historic
fffff  Found as keyword

***** Checking Analysis
fffff  Found as keyword

***** Checking Content Analysis
fffff  Found as keyword

***** Checking Enrichment
fffff  Found as keyword

***** Checking Organizing
fffff  Found as keyword

***** Checking document sharing
fffff  Found as keyword

***** Checking Annotation
fffff  Found as keyword

***** Checking virtual reality
fffff  Found as keyword

***** Checking visualization
fffff  Found as keyword

***** Checking Visualization
fffff  Found as keyword

***** Checking Discovering
fffff  Found a

iiiii  No Items found for provenance
Changing the property:  "type", from  "type": {'code': 'keyword', 'label': 'Keyword', 'type': 'concept', 'groupName': 'Categorisation', 'hidden': False, 'ord': 18, 'allowedVocabularies': [{'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'Keywords from SSHOC MP', 'closed': False}]} 
to
 "type": {'code': 'activity'}", in item with pid: "tools/SIU1nO"
(Log info: current version is: 28230)

Changing the property:  "concept", from  "concept": {'code': 'Capture', 'vocabulary': {'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'Keywords from SSHOC MP', 'closed': False}, 'label': 'Capture', 'notation': 'Capture', 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/Capture', 'candidate': True} 
to
 "concept": {'code': 'capturing', 'vocabulary': {'code': 'tadirah2'},

Changing the property:  "concept", from  "concept": {'code': 'Organizing', 'vocabulary': {'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'Keywords from SSHOC MP', 'closed': False}, 'label': 'Organizing', 'notation': 'Organizing', 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/Organizing', 'candidate': True} 
to
 "concept": {'code': 'organizing', 'vocabulary': {'code': 'tadirah2'}, 'uri': 'https://vocabs.dariah.eu/tadirah/organizing'}", in item with pid: "tools/oEYQGt"
(Log info: current version is: 28566)


 *** Running in DEBUG mode, Marketplace dataset not updated. *** 

Changing the property:  "type", from  "type": {'code': 'activity', 'label': 'Activity', 'type': 'concept', 'groupName': 'Categorisation', 'hidden': False, 'ord': 17, 'allowedVocabularies': [{'code': 'tadirah2', 'scheme': 'https://vocabs.dariah.eu/tadirah/', 'namespace': 'https://vocabs.dariah.eu/tadirah/', 'label': '

...


Changing the property:  "concept", from  "concept": {'code': 'crowdsourcing', 'vocabulary': {'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'Keywords from SSHOC MP', 'closed': False}, 'label': 'crowdsourcing', 'notation': 'crowdsourcing', 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/crowdsourcing', 'candidate': True} 
to
 "concept": {'code': 'crowdsourcing', 'vocabulary': {'code': 'tadirah2'}, 'uri': 'https://vocabs.dariah.eu/tadirah/crowdsourcing'}", in item with pid: "trainingMaterials/5lnjGj"
(Log info: current version is: 40759)


 *** Running in DEBUG mode, Marketplace dataset not updated. *** 

Changing the property:  "type", from  "type": {'code': 'keyword', 'label': 'Keyword', 'type': 'concept', 'groupName': 'Categorisation', 'hidden': False, 'ord': 18, 'allowedVocabularies': [{'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': '


Changing the property:  "concept", from  "concept": {'code': 'text', 'vocabulary': {'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'Keywords from SSHOC MP', 'closed': False}, 'label': 'text', 'notation': '', 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/text', 'candidate': False} 
to
 "concept": {'code': 'text', 'vocabulary': {'code': 'iana-media-type'}, 'uri': 'https://vocabs.sshopencloud.eu/vocabularies/media-type/text'}", in item with pid: "datasets/pd2HpN"
(Log info: current version is: 12551)


 *** Running in DEBUG mode, Marketplace dataset not updated. *** 

Changing the property:  "type", from  "type": {'code': 'object-format', 'label': 'Object format', 'type': 'concept', 'groupName': 'Categorisation', 'hidden': False, 'ord': 23, 'allowedVocabularies': [{'code': 'iana-media-type', 'scheme': 'https://vocabs.sshopencloud.eu/vocabularies/media-type/mediaTypeScheme', 'namespace':


Changing the property:  "concept", from  "concept": {'code': 'spatialAnalysis', 'vocabulary': {'code': 'tadirah2', 'scheme': 'https://vocabs.dariah.eu/tadirah/', 'namespace': 'https://vocabs.dariah.eu/tadirah/', 'label': 'TaDiRAH 2', 'closed': True}, 'label': 'Spatial Analysis', 'notation': '', 'uri': 'https://vocabs.dariah.eu/tadirah/spatialAnalysis', 'candidate': False} 
to
 "concept": {'code': 'spatialAnalysis', 'vocabulary': {'code': 'tadirah2'}, 'uri': 'https://vocabs.dariah.eu/tadirah/spatialAnalysis'}", in item with pid: "datasets/EpZ2mv"
(Log info: current version is: 40423)


 *** Running in DEBUG mode, Marketplace dataset not updated. *** 

Changing the property:  "type", from  "type": {'code': 'keyword', 'label': 'Keyword', 'type': 'concept', 'groupName': 'Categorisation', 'hidden': False, 'ord': 18, 'allowedVocabularies': [{'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'K


Changing the property:  "concept", from  "concept": {'code': 'linguistics', 'vocabulary': {'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'Keywords from SSHOC MP', 'closed': False}, 'label': 'linguistics', 'notation': '', 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/linguistics', 'candidate': False} 
to
 "concept": {'code': '6020', 'vocabulary': {'code': 'discipline'}, 'uri': 'https://vocabs.acdh.oeaw.ac.at/oefosdisciplines/6020'}", in item with pid: "datasets/Q53a9o"
(Log info: current version is: 12469)


 *** Running in DEBUG mode, Marketplace dataset not updated. *** 

Changing the property:  "type", from  "type": {'code': 'keyword', 'label': 'Keyword', 'type': 'concept', 'groupName': 'Categorisation', 'hidden': False, 'ord': 18, 'allowedVocabularies': [{'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/ssh


Changing the property:  "concept", from  "concept": {'code': 'linguistics', 'vocabulary': {'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/sshoc-keyword/', 'label': 'Keywords from SSHOC MP', 'closed': False}, 'label': 'linguistics', 'notation': '', 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/linguistics', 'candidate': False} 
to
 "concept": {'code': '6020', 'vocabulary': {'code': 'discipline'}, 'uri': 'https://vocabs.acdh.oeaw.ac.at/oefosdisciplines/6020'}", in item with pid: "datasets/dnEWZ8"
(Log info: current version is: 12672)


 *** Running in DEBUG mode, Marketplace dataset not updated. *** 

Changing the property:  "type", from  "type": {'code': 'keyword', 'label': 'Keyword', 'type': 'concept', 'groupName': 'Categorisation', 'hidden': False, 'ord': 18, 'allowedVocabularies': [{'code': 'sshoc-keyword', 'scheme': 'https://vocabs.dariah.eu/sshoc-keyword/Schema', 'namespace': 'https://vocabs.dariah.eu/ssh

In [ ]:
selectedItems.count()